# Juypter Notebook used in Google Colab 
This is notebook contains cells that used to test the exact py script that is used in Nautilus.  Since Nautilus does not have a simple way to do debugging, it is better to use a different service to do the debugging.

Import Libraries into Google Colab

In [ ]:
#!pip install -U bitsandbytes
!pip install torch
#!pip install peft
#!pip install transformers
!pip install qwen-vl-utils
!pip install -q "transformers==4.57.1" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.0" "sentencepiece" "einops" "timm"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
import torch
from transformers import AutoModelForImageTextToText, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from qwen_vl_utils import process_vision_info
import json
# import os

MODEL_PATH = "/content/drive/MyDrive/VLM_MODELS/Qwen2.5-VL-7B-Instruct"
DATASET_DIR = "/content/drive/MyDrive/model_datasets/dataset2"
OUTPUT_DIR = "/content/drive/MyDrive/output/waste_detection"
TRAINING_DATA = "/content/drive/MyDrive/model_datasets/train2.jsonl"
VAL_DATA = "/content/drive/MyDrive/model_datasets/valid2.jsonl"


print("Checking paths...\n")
print("Model path exists:", os.path.exists(MODEL_PATH))
print("Training data exists:", os.path.exists(TRAINING_DATA))
print("Validation data exists:", os.path.exists(VAL_DATA))
print("Output folder exists:", os.path.exists(OUTPUT_DIR))

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print("Output directory created.")

In [ ]:
import torch
from qwen_vl_utils import process_vision_info

class WasteDetectionDataset(torch.utils.data.Dataset):
    """Custom dataset that doesn't use Arrow/Datasets library"""
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {"messages": self.data[idx]["messages"]}

class WasteDetectionDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        texts, batch_images, batch_videos = [], [], []

        for f in features:
            msgs = f["messages"]
            # Apply chat template
            text = self.processor.apply_chat_template(
                msgs, tokenize=False, add_generation_prompt=False
            )
            texts.append(text)

            imgs, vids = process_vision_info(msgs)
            batch_images.append(imgs or [])
            batch_videos.append(vids or [])

        images = None if all(len(x) == 0 for x in batch_images) else batch_images
        videos = None if all(len(x) == 0 for x in batch_videos) else batch_videos

        batch = self.processor(
            text=texts,
            images=images,
            max_length=2048,   #None, It was either 512 or was too large?
            videos=videos,
            padding=True, # padding=True, truncation=True,
            truncation = False,
            return_tensors="pt"
        )

        # Get labels from input_ids
        labels = batch["input_ids"].clone()

        # For Qwen models, we need to mask everything except assistant response
        # The key is to find where assistant response tokens start
        for idx in range(len(features)):
            # Get the full sequence
            input_ids_list = batch["input_ids"][idx].tolist()

            # Mask everything first
            labels[idx] = -100

            # Find assistant content from original messages
            msgs = features[idx]["messages"]
            assistant_msg = None
            for msg in msgs:
                if msg["role"] == "assistant":
                    assistant_msg = msg["content"]
                    break

            if assistant_msg:
                # Tokenize just the assistant response
                asst_tokens = self.processor.tokenizer.encode(
                    assistant_msg,
                    add_special_tokens=False
                )

                # Find these tokens in the full sequence (from the end)
                seq_len = (batch["attention_mask"][idx] == 1).sum().item()

                # Match from the end backwards
                asst_len = len(asst_tokens)
                if asst_len > 0 and asst_len < seq_len:
                    # Check if the tokens match at the end
                    expected_start = seq_len - asst_len

                    # Unmask the assistant portion
                    labels[idx, expected_start:seq_len] = batch["input_ids"][idx, expected_start:seq_len]

        batch["labels"] = labels
        return batch

print("Collator defined - matching from end of sequence")


In [ ]:
# QLoRA 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with 8-bit quantization
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True
)

# Load processor
processor = AutoProcessor.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    local_files_only=True
)
print(f"Model loaded with 8-bit quantization: {type(model).__name__}")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Enable gradient checkpointing
model.enable_input_require_grads()
model.config.use_cache = False

print("8-bit QLoRA configured")

In [ ]:
import json
# Load training data
print("Loading training data...")
train_data = []
with open(TRAINING_DATA, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

train_dataset = WasteDetectionDataset(train_data)
print(f"Loaded {len(train_dataset)} training samples")

# Load validation data
print("Loading validation data...")
val_data = []
#with open(VALIDATION_DATA, 'r') as f:
with open(VAL_DATA, 'r') as f:
    for line in f:
        val_data.append(json.loads(line))

val_dataset = WasteDetectionDataset(val_data)
print(f"Loaded {len(val_dataset)} validation samples")

# Load test data (for later evaluation)
#print("Loading test data...")
#test_data = []
#with open(TEST_DATA, 'r') as f:
 #   for line in f:
 #       test_data.append(json.loads(line))

# test_dataset = WasteDetectionDataset(test_data)
# print(f"✓ Loaded {len(test_dataset)} test samples")

print(f"\n{'='*70}")
print(f"Dataset Summary:")
print(f"  Train:      {len(train_dataset)} examples")
print(f"  Validation: {len(val_dataset)} examples")
# print(f"  Test:       {len(test_dataset)} examples")
# print(f"  Total:      {len(train_dataset) + len(val_dataset) + len(test_dataset)} examples")
print(f"  Total:      {len(train_dataset) + len(val_dataset)} examples")
print(f"{'='*70}")


In [ ]:
print("Configuring TrainingArguments...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,  # originally 5

    # Batch sizes (for 8-bit LoRA), DO NOT USE WITH L4!
    # per_device_train_batch_size=2,
    # per_device_eval_batch_size=2,
    # gradient_accumulation_steps=4,

    # Sizes for QLoRA
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,

    # Learning rate
    learning_rate=2e-4,
    warmup_steps=100,

    # Evaluation
    eval_strategy="steps",
    eval_steps=50,

    # Logging
    logging_steps=10,
    logging_dir=f"{OUTPUT_DIR}/logs",

    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Optimization
    gradient_checkpointing=False,   # Was original set to true, but was sending a Warning in the last cell
    # optim="adamw_8bit",  # 8-bit optimizer, use with LoRA
    optim="adamw_torch",  # Should be ok with QLoRA?

    # Memory
    dataloader_pin_memory=True,
    remove_unused_columns=False,

    # Precision
    bf16=(torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8),
    fp16=not (torch.cuda.get_device_capability()[0] >= 8),

    # Reporting
    report_to="none",
)

print("TrainingArguments ready.\n")


In [ ]:
# Cell 9
print("Initializing Trainer...")

from transformers import Trainer

data_collator = WasteDetectionDataCollator(processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)
print("Trainer initialized.\n")

In [ ]:
print("="*70)
print("FINAL COLLATOR VERIFICATION")
print("="*70)

# Test two samples
test_batch = data_collator([train_dataset[0], train_dataset[1]])

print(f"\nBatch size: {test_batch['input_ids'].shape[0]}")
print(f"Sequence length: {test_batch['input_ids'].shape[1]}")

for i in range(test_batch['input_ids'].shape[0]):
    total = test_batch["labels"][i].numel()
    non_masked = (test_batch["labels"][i] != -100).sum().item()

    msgs = [train_dataset[0], train_dataset[1]][i]["messages"]
    expected = 0
    for msg in msgs:
        if msg["role"] == "assistant":
            expected = len(processor.tokenizer.encode(
                msg["content"], add_special_tokens=False
            ))
            break

    print(f"\nExample {i+1}:")
    print(f"  Learning from: {non_masked}/{total} tokens ({non_masked/total*100:.1f}%)")
    print(f"  Expected: {expected} tokens")
    print(f"  Match: {'✓ PERFECT' if abs(expected - non_masked) < 5 else '✗ MISMATCH'}")

print("\n" + "="*70)
print("COLLATOR STATUS: READY FOR TRAINING ✓")
print("="*70 + "\n")

In [ ]:
print(f"Training on {len(train_dataset)} examples")
print(f"Validating on {len(val_dataset)} examples")
print("Starting training...\n")

trainer.train()
print("\nSaving model and processor...")
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print(f"Training complete! Model saved to {OUTPUT_DIR}")

In [ ]:
from collections import defaultdict
# REVISE

by_step = defaultdict(dict)
for log in trainer.state.log_history:
    step = log.get("step")
    if step is None:
        continue
    for k, v in log.items():
        if k == "step":
            continue
        by_step[step][k] = v

print("Step\tTrain Loss\tEval Loss")
for step in sorted(by_step.keys()):
    train_loss = by_step[step].get("loss")
    eval_loss = by_step[step].get("eval_loss")
    if train_loss is None and eval_loss is None:
        continue
    print(f"{step}\t{train_loss}\t{eval_loss}")